In [2]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision

import pytorch_lightning as pl

from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader

from torchmetrics import Accuracy

from torchvision import transforms
from torchvision.datasets import CIFAR10

import numpy as np
from PIL import Image

import gradio as gr


c:\Users\Matyiko\Documents\Egyetem\Msc_2_felev\Melytanulas\ImageClassification\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class CIFAR10DataModule(pl.LightningDataModule):
    '''organize the data pipeline from accessing the data to loading it using PyTorch dataloaders'''

    #Initialization of the data_module
    def __init__(self, batch_size, data_dir: str = './data'):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform =  transforms.Compose([
            #Resizing the images to 224x224
            transforms.Resize(224),
            transforms.ToTensor(),
            #Normalization of the images
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        #Defining the 10 classes of the Cifar10 dataset
        self.num_classes = 10
        self.classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

    def prepare_data(self):
        # download the CIFAR-10 dataset
        torchvision.datasets.CIFAR10(self.data_dir, train=True, download=True)
        torchvision.datasets.CIFAR10(self.data_dir, train=False, download=True)

    # PyTorch dataset instances
    def setup(self, stage=None):
        #Using a 90-10 ratio on train-validation split
        if stage == 'fit' or stage is None:
            cifar_full = torchvision.datasets.CIFAR10(self.data_dir, train=True, download=True, transform=self.transform)
            self.cifar_train, self.cifar_val = random_split(cifar_full, [45000, 5000])
        #Using a different test set form the train and validation
        if stage == 'test' or stage is None:
            self.cifar_test =  torchvision.datasets.CIFAR10(self.data_dir, train=False, download=True, transform=self.transform)

    # dataloaders
    def train_dataloader(self):
        return DataLoader(self.cifar_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=self.batch_size)

In [4]:
class CIFARLitModel(pl.LightningModule):
    '''model architecture, training, testing and validation loops'''
    def __init__(self, pretrained, learning_rate=3e-4):
        super().__init__()

        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        #Using the Resnet-18 model with an option of being PreTrained or Not
        self.model  = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights='IMAGENET1K_V1' if pretrained else 'DEFAULT')
        
        self.model.fc = nn.Linear(in_features=512, out_features=10, bias=True)
        
        self.accuracy = Accuracy(task="multiclass", num_classes=10)

    def forward(self, x):
         return self.model(x)

    # train loop
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        # metric
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, logger=True)
        return loss

    # validation loop
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    # test loop
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        return loss

    # optimizers
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [5]:
model = CIFARLitModel(pretrained = True, learning_rate=1e-4 / 2)
# instantiate classes
dm = CIFAR10DataModule(batch_size=48)
dm.prepare_data()
dm.setup()
# Initialize Callbacks
from pathlib import Path

checkpoint_callback = pl.callbacks.ModelCheckpoint()
early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_acc", patience=3, verbose=False, mode="max")

if torch.cuda.is_available():
    accel = 'gpu'
else:
    accel = 'cpu'
trainer = pl.Trainer(
    accelerator=accel,
    max_epochs=10,
                     callbacks=[checkpoint_callback, early_stop_callback],
                    )
# Train the model
trainer.fit(model, dm, )
# Evaluate the model
trainer.test(dataloaders=dm.test_dataloader())

Using cache found in C:\Users\Matyiko/.cache\torch\hub\pytorch_vision_v0.10.0


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 3070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


...

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Restoring states from the checkpoint path at c:\Users\Matyiko\Documents\Egyetem\Msc_2_felev\Melytanulas\ImageClassification\build\lightning_logs\version_15\checkpoints\epoch=8-step=8442.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at c:\Users\Matyiko\Documents\Egyetem\Msc_2_felev\Melytanulas\ImageClassification\build\lightning_logs\version_15\checkpoints\epoch=8-step=8442.ckpt
c:\Users\Matyiko\Documents\Egyetem\Msc_2_felev\Melytanulas\ImageClassification\myenv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 209/209 [00:33<00:00,  6.16it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9438999891281128     │
│         test_loss         │    0.2068967968225479     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2068967968225479, 'test_acc': 0.9438999891281128}]

In [6]:
modelNotPretrained = CIFARLitModel(pretrained = False, learning_rate=1e-4 / 2)

# instantiate classes
dm = CIFAR10DataModule(batch_size=48)
dm.prepare_data()
dm.setup()
# Initialize Callbacks
from pathlib import Path

checkpoint_callback = pl.callbacks.ModelCheckpoint()
early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_acc", patience=3, verbose=False, mode="max")
trainer = pl.Trainer(max_epochs=10,
                     callbacks=[checkpoint_callback, early_stop_callback],
                    )
# Train the model
trainer.fit(modelNotPretrained, dm, )
# Evaluate the model
trainer.test(dataloaders=dm.test_dataloader())


Using cache found in C:\Users\Matyiko/.cache\torch\hub\pytorch_vision_v0.10.0


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type               | Params
------------------------------------------------
0 | model    | ResNet             | 11.2 M
1 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


...

Epoch 9: 100%|██████████| 938/938 [02:25<00:00,  6.46it/s, v_num=16, val_loss=0.191, val_acc=0.948]
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Restoring states from the checkpoint path at c:\Users\Matyiko\Documents\Egyetem\Msc_2_felev\Melytanulas\ImageClassification\build\lightning_logs\version_16\checkpoints\epoch=9-step=9380.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at c:\Users\Matyiko\Documents\Egyetem\Msc_2_felev\Melytanulas\ImageClassification\build\lightning_logs\version_16\checkpoints\epoch=9-step=9380.ckpt


Testing DataLoader 0: 100%|██████████| 209/209 [00:20<00:00, 10.14it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.940500020980835     │
│         test_loss         │    0.22119109332561493    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.22119109332561493, 'test_acc': 0.940500020980835}]

In [7]:
def preprocess_test_image(img):
    # Convert the image to a NumPy array
    img_array = np.array(img)

    # Make a copy of the array to avoid the resizing error
    img_array_copy = np.copy(img_array)

    # Resize the image to match the CIFAR10 dataset size (224x224 pixels)
    img_array_copy = Image.fromarray(img_array_copy)
    img_array_copy = img_array_copy.resize((224, 224))

    # Convert the image to a PyTorch tensor and apply normalization
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_tensor = transform(img_array_copy)

    # Add batch dimension (1, 3, 224, 224) to match the batch format used in the dataloader
    img_tensor = img_tensor.unsqueeze(0)

    return img_tensor

In [10]:
def image_classifier_pretrained(inp):
    model.eval()
    preprocessed_image = preprocess_test_image(inp)

    # Make predictions
    with torch.no_grad():
        output = model(preprocessed_image)

    # Convert the model output to probabilities using softmax
    probabilities = F.softmax(output[0], dim=0)

    # Create a formatted string with label names and corresponding probabilities
    result_str = {dm.classes[i]: float(probabilities[i]) for i in range(len(dm.classes))}

    return result_str

#interface_pretrained = gr.Interface(fn=image_classifier_pretrained, inputs="image", outputs="label", title="PRETRAINED")

def image_classifier_not_pretrained(inp):
    modelNotPretrained.eval()
    
    preprocessed_image = preprocess_test_image(inp)

    # Make predictions
    with torch.no_grad():
        output = modelNotPretrained(preprocessed_image)

    # Convert the model output to probabilities using softmax
    probabilities = F.softmax(output[0], dim=0)

    # Create a formatted string with label names and corresponding probabilities
    result_str = {dm.classes[i]: float(probabilities[i]) for i in range(len(dm.classes))}

    return result_str

#interface_not_pretrained = gr.Interface(fn=image_classifier_not_pretrained, inputs="image", outputs="text", title="NOT PRETRAINED")

with gr.Blocks() as demo:
    gr.Markdown(
    """
    # Hello!
    Try out the different models below!
    """)
    input = gr.Image()
    gr.Interface(fn=image_classifier_pretrained, inputs = input, outputs="label", title="PRETRAINED"),
    gr.Interface(fn=image_classifier_not_pretrained, inputs = input, outputs="label", title="NOT PRETRAINED")

if __name__ == "__main__":
    demo.launch(share=True)

# Launch both interfaces
#interface_pretrained.launch()
#interface_not_pretrained.launch()

Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
